In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import date

In [10]:
spark = SparkSession.builder.appName("E-commerce Transactions Analysis").getOrCreate()

In [11]:
schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("category", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("transaction_date", DateType(), True)
])

In [12]:
data = [
    ("T9", "U4", "P9", "Books", 30.0, date(2024, 4, 1)),
    ("T10", "U4", "P10", "Books", 40.0, date(2024, 4, 2)),
    ("T11", "U5", "P11", "Clothing", 200.0, date(2024, 5, 1)),
    ("T12", "U5", "P12", "Electronics", 150.0, date(2024, 5, 3)),
    ("T13", "U5", "P13", "Books", 50.0, date(2024, 5, 4)),
    ("T14", "U6", "P14", "Groceries", 20.0, date(2024, 6, 1)),
    ("T15", "U6", "P15", "Groceries", 30.0, date(2024, 6, 2)),
    ("T16", "U6", "P16", "Electronics", 100.0, date(2024, 6, 3))
]


In [13]:
df = spark.createDataFrame(data, schema)

In [14]:
spending_df = df.groupBy("user_id").agg(
    F.sum("amount").alias("total_spent"),
    F.avg("amount").alias("avg_transaction")
)

In [15]:
favorite_category_df = df.groupBy("user_id", "category") \
    .count() \
    .withColumn("rank", F.row_number().over(
        Window.partitionBy("user_id").orderBy(F.desc("count"))
    )) \
    .filter(F.col("rank") == 1) \
    .select("user_id", F.col("category").alias("favorite_category"))

In [16]:
result_df = spending_df.join(favorite_category_df, on="user_id", how="left")

In [17]:
result_df.show()

+-------+-----------+------------------+-----------------+
|user_id|total_spent|   avg_transaction|favorite_category|
+-------+-----------+------------------+-----------------+
|     U4|       70.0|              35.0|            Books|
|     U6|      150.0|              50.0|        Groceries|
|     U5|      400.0|133.33333333333334|         Clothing|
+-------+-----------+------------------+-----------------+

